In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import copy
import time
import os
import sys
sys.path.append('../../')

from data.processing import get_data

import models

from main.seir.fitting import single_fitting_cycle
from main.seir.forecast import get_forecast, forecast_all_trials, create_all_trials_csv, create_decile_csv_new
from main.seir.sensitivity import calculate_sensitivity_and_plot
from utils.generic.create_report import save_dict_and_create_report
from utils.generic.config import read_config
from utils.generic.enums import Columns
from utils.fitting.loss import Loss_Calculator
#from utils.generic.logging import log_wandb
from viz import plot_forecast, plot_top_k_trials, plot_ptiles

import yaml

In [ ]:
from os.path import exists, join, splitext

In [ ]:
config_filename = 'uncer.yaml'
config1 = read_config(config_filename)
config2 = read_config('default.yaml')

In [ ]:
percentile_dict1 = {}
percentile_dict2 = {}

In [ ]:
percentile_dict1['m1'] = single_fitting_cycle(**copy.deepcopy(config1['fitting']))
percentile_dict2['m1'] = single_fitting_cycle(**copy.deepcopy(config2['fitting']))

In [ ]:
for i in percentile_dict1['m1'].keys():
    print(i)

In [ ]:
percentile_dict1['m1']['df_loss']
from main.seir.forecast import _get_top_k_trials as topk
_,loss = topk(percentile_dict1['m1'], k = 10)
loss

In [ ]:
percentile_dict1['m1']['df_loss']
from main.seir.forecast import _get_top_k_trials as topk
_,loss = topk(percentile_dict2['m1'], k = 10)
loss

In [ ]:
percentile_dict2['m1']['df_loss']

In [ ]:
percentile_dict1['m1']['forecasts'] = {}
percentile_dict1['m1']['forecasts']['best'] = get_forecast(percentile_dict1, train_fit='m1', 
                                                           model=config1['fitting']['model'], 
                                                           days=config1['forecast']['forecast_days'])

percentile_dict1['m1']['trials_processed'] = forecast_all_trials(percentile_dict1, train_fit='m1', 
                                                                 model=config1['fitting']['model'], 
                                                                 forecast_days=config1['forecast']['forecast_days'])

In [ ]:
percentile_dict2['m1']['forecasts'] = {}
percentile_dict2['m1']['forecasts']['best'] = get_forecast(percentile_dict2, train_fit='m1', 
                                                           model=config2['fitting']['model'], 
                                                           days=config2['forecast']['forecast_days'])

percentile_dict2['m1']['trials_processed'] = forecast_all_trials(percentile_dict2, train_fit='m1', 
                                                                 model=config2['fitting']['model'], 
                                                                 forecast_days=config2['forecast']['forecast_days'])

In [ ]:
uncertainty_args1 = {'predictions_dict': percentile_dict1, **config1['uncertainty']['uncertainty_params']}
uncertainty1 = config1['uncertainty']['method'](**uncertainty_args1)

In [ ]:
uncertainty_args2 = {'predictions_dict': percentile_dict2, **config2['uncertainty']['uncertainty_params']}
uncertainty2 = config2['uncertainty']['method'](**uncertainty_args2)

In [ ]:
print("MCMC-",uncertainty1.p_val)
print("BO-",uncertainty2.p_val)

In [ ]:
uncertainty_forecasts1 = uncertainty1.get_forecasts()
uncertainty_forecasts2 = uncertainty2.get_forecasts()

In [ ]:
mcmc = []
bo = []
key = []
for i in uncertainty_forecasts1.keys():
    key.append(i)
    mcmc.append(uncertainty_forecasts1[i]['df_loss_perc'].mean())
    bo.append(uncertainty_forecasts2[i]['df_loss_perc'].mean())

In [ ]:
np.greater(mcmc,bo).sum()/len(mcmc)

In [ ]:
data = {"mcmc":mcmc,"bo":bo}
import pandas as pd
import seaborn as sns
import matplotlib.patches as mpatches

In [ ]:
plt.figure(figsize= (15,7))
mc, = plt.plot(key,mcmc,markersize=12,label='MCMC',color='b')
plt.scatter(key,mcmc,color='b')
b, = plt.plot(key,bo,markersize=12,label = 'BO',color='r')
plt.scatter(key,bo,color='r')
plt.legend(handles = [mc,b])
plt.xlabel('Percentiles')
plt.ylabel('Loss in Total compartment')

In [ ]:
np.linspace(0,100,40)